# VENDING MACHINE

## Welcome to my Vending machine!

Please feel free to use it by selecting **Run All** :)

In [179]:
# Generate data structures and variables

# Declaring global variables
choice = " "
#totPayment = 0
payment = 0

# Generate data structures
# TODO: MAKE THIS READ FROM FILE INPUT 

productDict = {"A1": {"Coffee": 0}, "A2": {"Tea": 5}, "B1": {"Apple Juice": 10}, "B2": {"Orange Juice": 10}}
extraIngredients = {"Sugar": 20, "Ice": 10}

In [180]:
# Function to display nested dictionary of products and keys

def displayProducts():
    """
    Displays Product list and if product is unavailable.
    """
    print("ITEMS AVAILABLE: \n")

    for product_ID, product_name in productDict.items(): 
        for key in product_name:
            if product_name[key] == 0: # Message if out of stock
                message = "NOT AVAILABLE"
            else:
                message = " "
            print(product_ID + "    " + key + " " + message)

# Display Home page of Vending Machine to allow choice of either user or administrator access

def displayVendingHome():
    """
    Function to display main prompt page.
    """
    print("Please select an option from below.")
    print()
    
    print("A    User Access")
    print("B    Admin Access")
    print("C    Turn Off") # TODO: USER INPUT CLOSES THE MACHINE DOWN

In [181]:
# Function to handle coin input from user
#def validateCoin():
#    """
#    Validates that coins inputted are legal tender values
#    """


def coinInput(price):
    """
    Allow for users to input their coins. Shows a total of how much has been paid 
    and how much is still remaining to be paid.
    """

    totPayment = 0
    coinFlag = True
    validCoinInput = [2.00, 1.00, 0.50, 0.20, 0.10, 0.05]
        
    while totPayment < price:
        payment = float(input("Please deposit your coin: "))

        while coinFlag == True:
            if payment in validCoinInput:
                coinFlag = True
                
                totPayment += payment
                remaining = price - totPayment
                print("Total: $" + "%.2f" % totPayment)
                if price > totPayment:
                    print("Please insert: $" + "%.2f" % remaining)
                    break
                elif price < totPayment:
                    refund = totPayment - price
                    print("Refund due...")
                    print("Dispensing: $" + "%.2f" % refund)
                    break
            else:
                coinFlag = False
                print("Please only enter legal tender.")
    print("Thank you. Payment processing...")

In [182]:
# Main Program
displayVendingHome()

choice = input("Selection: ").upper()
print()
if choice == "A":
    print("User Mode Active...")

    displayProducts()
    choice = input("Please enter product code: ").upper()

    if choice == "A1":
        dictList = productDict["A1"].keys()
        price = 3.50
        print(*dictList, sep ='', end=" ") # Print item from deconstructed list with price
        print("$" + str(price))

        coinInput(price)
    elif choice == "A2":
        dictList = productDict["A2"].keys()
        price = 2.00
        print(*dictList, sep ='', end=" ")
        print("$" + str(price))
    elif choice == "B1":
        dictList = productDict["B1"].keys()
        price = 4.00
        print(*dictList, sep ='', end=" ")
        print("$" + str(price))
    elif choice == "B2":
        dictList = productDict["B2"].keys()
        price = 4.00
        print(*dictList, sep ='', end=" ")
        print("$" + str(price))
    else:
        print("Error incorrect input. Please re-enter product code.")

elif choice == "B":
    print("Admin Access Menu:")
    # TODO: ADD SYSTEM STATUS MESSAGE SAYING MACHINE ONLY AVAILABLE FOR ADMINISTRATORS
    # TODO: ADD ADMIN ACCESS MENU AND PROMPTS
elif choice == "C":
    print("Complete me...")
    # TODO: ADD CODE TO CLOSE MACHINE
else:
    print("Incorrect input.")

Please select an option from below.

A    User Access
B    Admin Access
C    Turn Off

User Mode Active...
ITEMS AVAILABLE: 

A1    Coffee NOT AVAILABLE
A2    Tea  
B1    Apple Juice  
B2    Orange Juice  
Coffee $3.5
Total: $2.00
Please insert: $1.50
Total: $3.00
Please insert: $0.50
Please only enter legal tender.


ValueError: could not convert string to float: ''